In [ ]:
import pandas as pd

# Read the first CSV file into a DataFrame
df1 = pd.read_csv('/content/merged_Oneplus.csv')

# Read the second CSV file into another DataFrame
df2 = pd.read_csv('/content/merged_Realme narzo.csv')
# Read the second CSV file into another DataFrame
df3 = pd.read_csv('/content/merged_Redmi Note 11.csv')
df4 = pd.read_csv('/content/merged_Redmi10.csv')
df5 = pd.read_csv('/content/merged_Redmi9.csv')
df6 = pd.read_csv('/content/merged_samsung.csv')
df7 = pd.read_csv('/content/merged_vivo.csv')
df8 = pd.read_csv('/content/oppo_merged_file.csv')
# Merge the two DataFrames vertically
merged_df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8], axis=0)

# Write the merged DataFrame to a new CSV file if needed
merged_df.to_csv('merged_data.csv', index=False)


In [ ]:
import pandas as pd
merged_df=pd.read_csv('/content/merged_data.csv')
df3=merged_df.copy()
df3

,Product Name,Cleaned_Review,rating,Sentiment,text_length,Predicted Label
0,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",awesome phone cameradisplaytouchselfi canperfo...,5.0,1,388.0,performance
1,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",mid ranger flagship get mobile awesome deal ra...,5.0,1,289.0,camera
2,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",average phone keep average expectations write ...,3.0,1,287.0,performance
3,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",good choice upgrade nord ce decent choice some...,5.0,1,284.0,display
4,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",decent package non gamers everything thing dec...,4.0,1,278.0,display
...,...,...,...,...,...,...
8482,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",think old refurbish phone give screen not work...,1.0,0,11.0,display
8483,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",not anything dislike display always go blanke ...,2.0,0,11.0,display
8484,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",opo display quality problem not satisfy not sa...,2.0,0,11.0,display
8485,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",handset not find handset miss packageso sad bo...,5.0,1,11.0,display


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8487 entries, 0 to 8486
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Product Name     8487 non-null   object 
 1   Cleaned_Review   8487 non-null   object 
 2   rating           8487 non-null   float64
 3   Sentiment        8487 non-null   int64  
 4   text_length      8487 non-null   float64
 5   Predicted Label  8487 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 398.0+ KB


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, mean_squared_error
from textblob import TextBlob

# Split the data into training and testing sets
X = df3['Cleaned_Review']  # Text data
y_sentiment = df3['Sentiment']  # Positive or Negative labels
y_rating = df3['rating']  # Numeric ratings

X_train, X_test, y_sentiment_train, y_sentiment_test, y_rating_train, y_rating_test = train_test_split(X, y_sentiment, y_rating, test_size=0.2, random_state=42)

# Create TF-IDF vectors from the text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a sentiment analysis model (e.g., Multinomial Naive Bayes)
sentiment_classifier = MultinomialNB()
sentiment_classifier.fit(X_train_tfidf,y_sentiment_train)

# Predict sentiment on the test set
y_sentiment_pred = sentiment_classifier.predict(X_test_tfidf)

# Evaluate the sentiment analysis model
sentiment_accuracy = accuracy_score(y_sentiment_test, y_sentiment_pred)

# Calculate sentiment scores using TextBlob for the entire dataset
df3['textblob_sentiment'] = df3['Cleaned_Review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Normalize TextBlob sentiment scores to the [0, 1] scale
df3['normalized_sentiment'] = (df3['textblob_sentiment'] + 1) / 2

# Combine the model's predicted sentiment with TextBlob sentiment scores
df3['combined_sentiment'] = (df3['normalized_sentiment'] + (y_sentiment == 'positive').astype(int)) / 2

# Calculate the overall rating score on a scale of 5
df3['overall_rating'] = 5 *df3['combined_sentiment']

# Calculate the Mean Squared Error (MSE) for the ratings
rating_mse = mean_squared_error(y_rating_test,df3.loc[y_rating_test.index, 'overall_rating'])

# Print results
print(f'Sentiment Analysis Accuracy: {sentiment_accuracy}')
print(f'Mean Squared Error for Ratings: {rating_mse}')


Sentiment Analysis Accuracy: 0.8392226148409894
Mean Squared Error for Ratings: 4.222229951842408


In [ ]:
# Count the number of positive (1) and negative (0) reviews
num_positive_reviews = (y_sentiment_pred == 1).sum()
num_negative_reviews = (y_sentiment_pred == 0).sum()

# Calculate the total number of reviews
total_reviews = len(y_sentiment_pred)

# Calculate the percentage of positive and negative reviews
percentage_positive = (num_positive_reviews / total_reviews) * 100
percentage_negative = (num_negative_reviews / total_reviews) * 100

print(f'Percentage of Positive Reviews: {percentage_positive}%')
print(f'Percentage of Negative Reviews: {percentage_negative}%')

Percentage of Positive Reviews: 60.30624263839811%
Percentage of Negative Reviews: 39.69375736160188%


In [ ]:
# Calculate the rating out of 5
rating_out_of_5 = (percentage_positive / 100) * 5

# You can choose to use the negative percentage to adjust the rating
# For example, if negative reviews should reduce the rating:
# rating_out_of_5 -= (percentage_negative / 100) * 5

# Round the rating to the nearest decimal place, if desired
rounded_rating = round(rating_out_of_5, 1)

# Print the rating
print(f'Rating out of 5: {rounded_rating}')

Rating out of 5: 3.0


In [ ]:
# Define the unique values in the "Predicted Label" column
unique_labels = df3['Predicted Label'].unique()

# Create dictionaries to store sentiment counts for each label
label_sentiments = {}

# Iterate through each unique label
for label in unique_labels:
    # Filter the DataFrame for rows where the "Predicted Label" matches the current label
    filtered_df = df3[df3['Predicted Label'] == label]

    # Count positive and negative sentiments in the filtered data
    num_positive_sentiments = (filtered_df['Sentiment'] == 1).sum()
    num_negative_sentiments = (filtered_df['Sentiment'] == 0).sum()

    # Store the sentiment counts in the dictionary
    label_sentiments[label] = {
        1: num_positive_sentiments,
        0: num_negative_sentiments
    }

# Print the results
for label in unique_labels:
    print(f'Label: {label}')
    print(f'Positive Sentiments: {label_sentiments[label][1]}')
    print(f'Negative Sentiments: {label_sentiments[label][0]}')
    print()


Label: performance
Positive Sentiments: 599
Negative Sentiments: 857

Label: camera
Positive Sentiments: 1292
Negative Sentiments: 1142

Label: display
Positive Sentiments: 673
Negative Sentiments: 1165

Label: value for money
Positive Sentiments: 1931
Negative Sentiments: 140

Label: battery
Positive Sentiments: 375
Negative Sentiments: 313



In [ ]:
import pandas as pd

# Load your DataFrame (assuming you have already loaded your data)
#df2 = pd.read_csv('/content/classification_results.csv')

# Define the unique values in the "Predicted Label" column
unique_labels = df3['Predicted Label'].unique()

# Create dictionaries to store sentiment percentages for each label
label_sentiments = {}

# Iterate through each unique label
for label in unique_labels:
    # Filter the DataFrame for rows where the "Predicted Label" matches the current label
    filtered_df = df3[df3['Predicted Label'] == label]

    # Count positive and negative sentiments in the filtered data
    num_positive_sentiments = (filtered_df['Sentiment'] == 1).sum()
    num_negative_sentiments = (filtered_df['Sentiment'] == 0).sum()

    total_sentiments = len(filtered_df)

    # Calculate the percentages
    if total_sentiments > 0:
        positive_percentage = (num_positive_sentiments / total_sentiments) * 100
        negative_percentage = (num_negative_sentiments / total_sentiments) * 100
    else:
        positive_percentage = 0
        negative_percentage = 0

    # Store the sentiment percentages in the dictionary
    label_sentiments[label] = {
        'Positive Percentage': positive_percentage,
        'Negative Percentage': negative_percentage
    }

# Print the results
for label in unique_labels:
    print(f'Label: {label}')
    print(f'Positive Percentage: {label_sentiments[label]["Positive Percentage"]:.2f}%')
    print(f'Negative Percentage: {label_sentiments[label]["Negative Percentage"]:.2f}%')
    print()


Label: performance
Positive Percentage: 41.14%
Negative Percentage: 58.86%

Label: camera
Positive Percentage: 53.08%
Negative Percentage: 46.92%

Label: display
Positive Percentage: 36.62%
Negative Percentage: 63.38%

Label: value for money
Positive Percentage: 93.24%
Negative Percentage: 6.76%

Label: battery
Positive Percentage: 54.51%
Negative Percentage: 45.49%



In [ ]:
# Create a function to calculate ratings based on sentiment percentages
def calculate_rating(positive_percentage, negative_percentage, rating_scale=5):
    # The formula to calculate the rating based on sentiment percentages
    rating = (positive_percentage / 100) * rating_scale

    # Optionally, you can adjust the rating based on the negative sentiment percentage
    # For example, reduce the rating if there is a significant negative sentiment
    # rating -= (negative_percentage / 100) * rating_scale

    return rating

# Create a dictionary to store ratings for each label
label_ratings = {}

# Calculate and store ratings based on sentiment percentages for each label
for label in unique_labels:
    positive_percentage = label_sentiments[label]['Positive Percentage']
    negative_percentage = label_sentiments[label]['Negative Percentage']

    rating = calculate_rating(positive_percentage, negative_percentage)

    label_ratings[label] = rating

# Print the ratings
for label in unique_labels:
    print(f'Label: {label}')
    print(f'Label: {label}')
    print(f'Positive Percentage: {label_sentiments[label]["Positive Percentage"]:.2f}%')
    print(f'Negative Percentage: {label_sentiments[label]["Negative Percentage"]:.2f}%')

    print(f'Rating: {label_ratings[label]:.2f} out of 5')
    print()


Label: performance
Label: performance
Positive Percentage: 41.14%
Negative Percentage: 58.86%
Rating: 2.06 out of 5

Label: camera
Label: camera
Positive Percentage: 53.08%
Negative Percentage: 46.92%
Rating: 2.65 out of 5

Label: display
Label: display
Positive Percentage: 36.62%
Negative Percentage: 63.38%
Rating: 1.83 out of 5

Label: value for money
Label: value for money
Positive Percentage: 93.24%
Negative Percentage: 6.76%
Rating: 4.66 out of 5

Label: battery
Label: battery
Positive Percentage: 54.51%
Negative Percentage: 45.49%
Rating: 2.73 out of 5



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
import joblib

# Example: Train and save the model
texts = ["This is a positive example.", "This is a negative example.", "This is another positive example."]
labels = ["positive", "negative", "positive"]

model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(texts, labels)

# Save the model to a file
joblib.dump(model, "multinomialnb_model.pkl")

['multinomialnb_model.pkl']

In [ ]:
import joblib

# Load the model from the file
loaded_model = joblib.load("multinomialnb_model.pkl")

# Example: Use the loaded model for prediction
text_to_predict = "This is a new text for prediction."
prediction = loaded_model.predict([text_to_predict])

print("Prediction:", prediction[0])


Prediction: positive


In [ ]:
# Assuming X_test contains your test set text data
predictions = loaded_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


ValueError: ignored

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from PIL import Image

image = Image.open('/content/Sentimentlow.jpg')

st.image(image, caption='Sentimentlow')

2023-11-08 10:37:41.326 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:
import os
current_directory = os.getcwd()
print(f"Current Working Directory: {current_directory}")

Current Working Directory: /content


In [ ]:
import pickle

# Save your model to a file using pickle
model = MultinomialNB()  # Replace with your actual trained model
tfidf_vectorizer = TfidfVectorizer()
model_file_path = '/content/sentiment_model.pkl'  # Provide the full file path for the model
tfidf_vectorizer_file_path = '/content/tfidf_vectorizer.pkl'  # Provide the full file path for the TF-IDF vectorizer

with open(model_file_path, 'wb') as model_file:
    pickle.dump(model, model_file)

with open(tfidf_vectorizer_file_path, 'wb') as tfidf_file:
    pickle.dump(tfidf_vectorizer, tfidf_file)

# Load the model and TF-IDF vectorizer from the saved files
with open(model_file_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open(tfidf_vectorizer_file_path, 'rb') as tfidf_file:
    loaded_tfidf_vectorizer = pickle.load(tfidf_file)

